In [2]:
from cdFBA.processes.dfba import DFBA, UpdateEnvironment, Chemostat, Injector, WaveFunction
from matplotlib import pyplot as plt

from cdFBA.utils import DFBAconfig, model_from_file, get_objective_reaction, get_injector_spec, get_wave_spec, \
    get_chemo_spec, dfba_config
from cdFBA.utils import dfba_config_from_model, get_single_dfba_spec, dfba_config, environment_spec, initial_environment, initial_conditions
import cobra

In [6]:
name1 = "Escherichia coli K12"
model_file_1 = "iAF1260"
medium_type='minimal'
model_1 = model_from_file(model_file=model_file_1)
media_1 = DFBAconfig(model=model_1, medium_type=medium_type)
initial_counts = initial_conditions(model=model_1, biomass=0.1, medium_type=medium_type, name=name1)
initial_env = initial_environment(volume=1.0, initial_counts= initial_counts)
config_1 = dfba_config(
    model_file = model_file_1,
    name = name1,
    kinetics = media_1.kinetics,
    reaction_map = media_1.reaction_map,
    biomass_identifier = media_1.biomass_indentifier,
    bounds = None,
)

In [7]:
spec={}
spec[name1] = get_single_dfba_spec(config=config_1, interval=0.5)

In [9]:
spec['shared environment'] = initial_environment(volume=2, species_list=[name1])